<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/Hybrid%20LR-Bi-LSTM%20for%2010%20class%20classifications%20based%20on%20UNSW-NB15%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid LR-Bi-LSTM for 10 class classifications based on UNSW-NB15 dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# Load dataset
dataset = pd.read_csv("dataset_P2.2.csv")

In [3]:
# Data preprocessing
y = dataset['Class'].values
X = dataset.drop(['Class'],axis=1)
X = X.values

In [4]:
# balance classes using SMOTE
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [5]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Hybrid combination of RF-DNN for 10 class classifications**

In [6]:
# train RF model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

RandomForestClassifier(random_state=42)

In [7]:
# train DNN model
input_dim = X.shape[1]
num_classes = len(np.unique(y))
dnn_model = Sequential()
dnn_model.add(Dense(128, input_dim=input_dim, activation='relu'))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dense(10, activation='softmax'))
dnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn_model.fit(X, pd.get_dummies(y), epochs=15, batch_size=32)

Epoch 1/15
17500/17500 [==============================] - 35s 2ms/step - loss: 1140431.2500 - accuracy: 0.1665
Epoch 2/15
17500/17500 [==============================] - 32s 2ms/step - loss: 15.6237 - accuracy: 0.1257
Epoch 3/15
17500/17500 [==============================] - 33s 2ms/step - loss: 7.9696 - accuracy: 0.1174
Epoch 4/15
17500/17500 [==============================] - 32s 2ms/step - loss: 3.6120 - accuracy: 0.1023
Epoch 5/15
17500/17500 [==============================] - 33s 2ms/step - loss: 3.4099 - accuracy: 0.1012
Epoch 6/15
17500/17500 [==============================] - 33s 2ms/step - loss: 7.7976 - accuracy: 0.1009
Epoch 7/15
17500/17500 [==============================] - 32s 2ms/step - loss: 2.3042 - accuracy: 0.1015
Epoch 8/15
17500/17500 [==============================] - 35s 2ms/step - loss: 2.3007 - accuracy: 0.1010
Epoch 9/15
17500/17500 [==============================] - 34s 2ms/step - loss: 2.3007 - accuracy: 0.1007
Epoch 10/15
17500/17500 [=======================

In [8]:
# combine RF and DNN models
def hybrid_model_predict(X):
    rf_pred = rf_model.predict(X)
    dnn_pred = np.argmax(dnn_model.predict(X), axis=-1)
    hybrid_pred = []
    for i in range(len(X)):
        if rf_pred[i] == dnn_pred[i]:
            hybrid_pred.append(rf_pred[i])
        else:
            hybrid_pred.append(np.random.choice([rf_pred[i], dnn_pred[i]]))
    return np.array(hybrid_pred)

# evaluate hybrid model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = hybrid_model_predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print('Hybrid model accuracy:', accuracy)

3500/3500 [==============================] - 4s 1ms/step
Hybrid model accuracy: 0.5526071428571429
